In [126]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
import requests
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [127]:
from selenium.webdriver.common.by import By

In [129]:
# driver = webdriver.Chrome()
# driver.get("https://www.glassdoor.com/Job/united-states-data-scientist-jobs-SRCH_IL.0,13_IN1_KO14,28_IP7.htm?includeNoSalaryJobs=true")

#     # Get element with tag name 'div'
# element = driver.find_element(By.CSS_SELECTOR, '[class="SVGInline modal_closeIcon"]').click()

# driver.maximize_window()

In [130]:
jobs = []

In [174]:
driver = webdriver.Chrome()
# driver.get("https://www.glassdoor.com/Job/united-states-data-scientist-jobs-SRCH_IL.0,13_IN1_KO14,28_IP7.htm?includeNoSalaryJobs=false")

num_jobs = 900
verbose = True
dup = 0
driver.get("https://www.glassdoor.com/Job/data-engineer-jobs-SRCH_KO0,13.htm")
# driver.get("https://www.glassdoor.com/Job/data-scientist-jobs-SRCH_KO0,14.htm?clickSource=searchBox")

while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.
        
        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(2)

        #Test for the "Sign Up" prompt and get rid of it.
#         try:
#             driver.find_element(By.CSS_SELECTOR, '[class="SVGInline modal_closeIcon"]').click()
#         except ElementClickInterceptedException:
#             pass

#         time.sleep(.1)

        try:
            driver.find_element(By.CSS_SELECTOR, '[class="SVGInline modal_closeIcon"]').click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements(By.CSS_SELECTOR, '[data-test="jobListing"]')  #jl for Job Listing. These are the buttons we're going to click.
        
              
        for job_button in job_buttons:  
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs) + "/ dup" + str(dup)))
            
            if len(jobs) >= num_jobs:
                break
                
            try:
                driver.find_element(By.CSS_SELECTOR, '[alt="Close"]')
            except:
                try:
                    driver.find_element(By.CSS_SELECTOR, '[alt="Close"]').click()  #clicking to the X.
                except NoSuchElementException:
                    pass
            
#             job_button.click()
            
            try:
                job_button.click()
            except NoSuchElementException:
                print(job_button.text)

      
            try:
                if driver.find_element(By.CSS_SELECTOR, '[alt="Close"]'):
                    time.sleep(3)
                    driver.find_element(By.CSS_SELECTOR, '[alt="Close"]').click()  #clicking to the X.
            except NoSuchElementException:
                pass

            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    try:
                        driver.find_element(By.CSS_SELECTOR, '[data-test="employerName"]')
                        company_name = driver.find_element(By.CSS_SELECTOR, '[data-test="employerName"]').text
                    except: 
                        company_name = driver.find_element(By.CSS_SELECTOR, '[class="d-flex justify-content-between"]').text
#                     try:
#                         company_name = driver.find_element(By.CSS_SELECTOR, '[class="d-flex justify-content-between"]').text
#                     except:
#                         NoSuchElementException
                    location = driver.find_element(By.CSS_SELECTOR, '[data-test="location"]').text
                    job_title = driver.find_element(By.CSS_SELECTOR, '[class="css-1vg6q84 e1tk4kwz4"]').text
                    driver.find_element(By.CSS_SELECTOR, '[class="css-t3xrds e856ufb4"]').click()
                    # time.sleep(10)
                    job_description = driver.find_element(By.CSS_SELECTOR, '[id="JobDescriptionContainer"]').text
                    # salary_estimate = driver.find_element(By.CSS_SELECTOR, '[class="css-1xe2xww e1wijj242"]').text
                    collected_successfully = True
#                     if len(job_title)<1:
#                         time.sleep(5)
#                         job_title = driver.find_element(By.CSS_SELECTOR, '[data-test="jobTitle"]').text
                except:
                    time.sleep(5)
                    
                    
            # class="d-flex justify-content-between"

            try:
#                 salary_element = WebDriverWait(driver, 10).until(
#                                     EC.presence_of_element_located((By.CSS_SELECTOR, "[class='css-1bluz6i e2u4hf13']")))
                salary_estimate = driver.find_element(By.CSS_SELECTOR, '[class="css-1bluz6i e2u4hf13"]').get_attribute("textContent")
#                 salary_estimate = salary_element.text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element(By.CSS_SELECTOR, '[data-test="detailRating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500])) #[:500]
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            
            #try:
            overview_name = driver.find_elements(By.CSS_SELECTOR, '[class="css-1taruhi e1pvx6aw1"]')
            overview_name_list = [element.text for element in overview_name]
            overview = driver.find_elements(By.CSS_SELECTOR, '[class="css-i9gxme e1pvx6aw2"]')
            overview_list = [element.text for element in overview]
#             for name, value in zip(overview_name_list, overview_list):
            if "Size" in overview_name_list:
                size = overview_list[overview_name_list.index("Size")]
            else:
                size = -1

            if "Founded" in overview_name_list:
                founded = overview_list[overview_name_list.index("Founded")]
            else:
                founded = -1

            if "Type" in overview_name_list:
                type_of_ownership = overview_list[overview_name_list.index("Type")]
            else:
                type_of_ownership = -1

            if "Industry" in overview_name_list:
                industry = overview_list[overview_name_list.index("Industry")]
            else:
                industry = -1

            if "Sector" in overview_name_list:
                sector = overview_list[overview_name_list.index("Sector")]
            else:
                sector = -1

            if "Revenue" in overview_name_list:
                revenue = overview_list[overview_name_list.index("Revenue")]
            else:
                revenue = -1

#                     founded = overview[1].text
#                     type_of_ownership = overview[2].text
#                     industry = overview[3].text
#                     sector = overview[4].text
#                     revenue = overview[5].text
#                 else:# NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
#                     headquarters = -1
#                     size = -1
#                     founded = -1
#                     type_of_ownership = -1
#                     industry = -1
#                     sector = -1
#                     revenue = -1

                
            if verbose:
                # print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                # print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
            new_dict = {"Job Title" : job_title,
                        "Salary Estimate" : salary_estimate,
                        "Job Description" : job_description,
                        "Rating" : rating,
                        "Company Name" : company_name,
                        "Location" : location,
                        #"Headquarters" : headquarters,
                        "Size" : size,
                        "Founded" : founded,
                        "Type of ownership" : type_of_ownership,
                        "Industry" : industry,
                        "Sector" : sector,
                        "Revenue" : revenue,}
            if new_dict not in jobs:
                jobs.append(new_dict)
            else:
                dup += 1
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element(By.CSS_SELECTOR, '[aria-label="Next"]').click()
            if dup > 200:
                break
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

Progress: 415/900/ dup0
Job Title: Data Engineer
Salary Estimate: $91,937 /yr (est.)
Job Description: Direct Connect Logistix is a nation-wide 3PL company headquartered in Indianapolis, Indiana. Since 2009, we have committed to our promise of delivering every shipment on time, every time - no matter what. That is why we hire the best people, inject tons of energy into our culture, and empower our people to make great things happen for our customers.
Job Summary
We are seeking a highly skilled and experienced data engineer to join our team. The ideal candidate will be responsible for designing, d
Rating: 3.7
Company Name: Direct Connect Logistix
3.7
Location: Indianapolis, IN
Size: 51 to 200 Employees
Founded: 2009
Type of Ownership: Company - Private
Industry: Taxi & Car Services
Sector: Transportation & Logistics
Revenue: $100 to $500 million (USD)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 415/900/ dup1
Job Title: Senior Engineer, Data Analytics
Salary Estimate: $

Job Title: Data Link Engineer
Salary Estimate: $110,000 /yr (est.)
Job Description: Candidate will need basic understanding of Python and basic experience is desired with operating common tactical data link radios/terminals/gateways/modems, MIDS LVT (Link 16), MIDS JTRS (Link 16), AN/ARC-210 (U/VHF), AN/PRC-117G (U/VHF). Should have some familiarity with the following data link waveforms and protocols:
Link 16, Variable Message Format (VMF) and the ability to develop and analyze data link messages (J-series messages, K-series messages). Ability to read and understand common mil
Rating: -1
Company Name: Avionics Test and Analysis Corporation
Location: Edwards AFB, CA
Size: -1
Founded: -1
Type of Ownership: -1
Industry: -1
Sector: -1
Revenue: -1
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 415/900/ dup10
Job Title: Senior Data Engineer
Salary Estimate: $108,655 /yr (est.)
Job Description: PURPOSE OF THE POSITION Advance Simmons Foods Business Intelligence capabilities, 

Job Title: Data Engineer
Salary Estimate: $93,854 /yr (est.)
Job Description: Job Family:
Data Engineering & Architecture Consulting

Travel Required:
None

Clearance Required:
Active Secret
What You Will Do:
The Candidate will provide full-stack development consulting to a Department of the Army client as part of a Digital Transformation team responsible for harnessing, cleaning, and analyzing large amounts of related data and providing an intuitive and user-friendly set of data visualization dashboards and applications based on this data, to foster a data-driven decisio
Rating: 3.7
Company Name: Guidehouse
3.7
Location: Huntsville, AL
Size: 1001 to 5000 Employees
Founded: 2018
Type of Ownership: Company - Private
Industry: Business Consulting
Sector: Management & Consulting
Revenue: Unknown / Non-Applicable
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 415/900/ dup20
Job Title: Pipeline Integrity Risk Assessment & Data Science Engineer(Hybrid Schedule)
Salary Estimat

Job Title: Blockchain Data Engineer, Blockchain Lab
Salary Estimate: $101,500 /yr (est.)
Job Description: The mission of FINRA’s Blockchain Lab (the “Lab”) is to serve as a central point within FINRA for the development of blockchain initiatives; to build capacity within the organization to develop technology solutions to oversee the activity of member firm’s blockchain related activities; to identify, analyze and assist with the deployment of offerings by third-party blockchain vendors to assist with FINRA’s regulatory functions; and to serve as resource within FINRA for expertise on matters relate
Rating: 4.1
Company Name: Finra
4.1
Location: Washington, DC
Size: 1001 to 5000 Employees
Founded: 2007
Type of Ownership: Nonprofit Organization
Industry: Investment & Asset Management
Sector: Financial Services
Revenue: $500 million to $1 billion (USD)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 415/900/ dup29
Job Title: Senior Data Engineer, Cloud Solutions Development

Job Title: Data Engineer
Salary Estimate: $87,500 /yr (est.)
Job Description: Job description
About the Role:Data and translation of data into information is core to Trailhead Biosystems strategy and business. You will work directly with our founding team to help develop a backend data infrastructure to both support Trailhead Biosystems internal bioinformaticians and platform as well as external-facing partnerships. You will have the opportunity to evaluate cutting edge data tools and be part of a team working in the space of computational biology.The ideal candidate will
Rating: 2.5
Company Name: Trailhead Biosystems Inc.
2.5
Location: Beachwood, OH
Size: 1 to 50 Employees
Founded: -1
Type of Ownership: Unknown
Industry: -1
Sector: -1
Revenue: Unknown / Non-Applicable
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 415/900/ dup38
Job Title: Data Engineer, Senior Consultant
Salary Estimate: $97,622 /yr (est.)
Job Description: Job Family:
Data Science Consulting

Travel R

...

Job Title: Sr Data Storage Engineer, remote
Salary Estimate: $102,499 /yr (est.)
Job Description: Overview:
Fairview Health Services is hiring a Sr Data Storage Engineer! The Sr Engineer role includes crafting, configuring, developing, integrating, testing, implementing, detailing and supporting data storage infrastructure environments. Our Storage Services team handles the building of services through a deep understanding of the application requirements. We work with Architecture and Operational teams to develop, apply and support Enterprise Storage and Backup Infrastructure processes, tool
Rating: 3.4
Company Name: Fairview Health Services
3.4
Location: Minneapolis, MN
Size: 10000+ Employees
Founded: -1
Type of Ownership: Nonprofit Organization
Industry: Health Care Services & Hospitals
Sector: Healthcare
Revenue: $1 to $5 billion (USD)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 455/900/ dup191
Job Title: Data Engineer Principal
Salary Estimate: $124,601 /yr (est.

Size: 1001 to 5000 Employees
Founded: 1969
Type of Ownership: Company - Private
Industry: Computer Hardware Development
Sector: Information Technology
Revenue: $1 to $5 billion (USD)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 455/900/ dup200
Job Title: Senior Engineer, Data Analytics
Salary Estimate: $114,341 /yr (est.)
Job Description: Work at ROCKET PHARMA and help cure rare diseases!

Rocket Pharma is a fully integrated, clinical-stage company advancing gene therapies with curative potential for multiple rare childhood diseases. Rocket places enormous value on people and considers team member progress and well-being to be as important as the progress of its pipeline. We are looking for hands-on team players who enjoy collaborating with colleagues. We share a passion for seeking gene therapy cures for devastating diseases and
Rating: 3.4
Company Name: Rocket Pharmaceuticals
3.4
Location: Cranbury, NJ
Size: 51 to 200 Employees
Founded: 2015
Type of Ownership: Compa

Size: 501 to 1000 Employees
Founded: 1966
Type of Ownership: Nonprofit Organization
Industry: Health Care Services & Hospitals
Sector: Healthcare
Revenue: $25 to $100 million (USD)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 455/900/ dup209
Job Title: Senior Data Platform Engineer
Salary Estimate: $111,922 /yr (est.)
Job Description: :
The Data Platform Engineer is a key resource in providing technical leadership and support on all the stages of Data Platform administration, development, and architecture implementation. We’re looking for an enthusiastic team member to engage and collaborate with leaders and team members from teams across our enterprise (including Server, Infrastructure, dozens of Development teams, and more), as we continue our transformation from a platform-based to a solution-based shop. This team member w
Rating: 2.7
Company Name: United Wholesale Mortgage
2.7
Location: Pontiac, MI
Size: 5001 to 10000 Employees
Founded: 1986
Type of Ownership: Com

Job Title: Sr Data Integration Engineer
Salary Estimate: $113,745 /yr (est.)
Job Description: Compeer Financial is seeking collaborative, innovative and dynamic professionals to be a part of our Top Workplace culture!
Tell me more about this opportunity.
Position Overview:
This position is responsible for the development, enhancement and effective operations of data movement between various systems using ETL methodologies. Responsible for leading company database development efforts and coordinating with business users, consultants/contractors, vendors and internal team members to maximi
Rating: 4.4
Company Name: Compeer Financial
4.4
Location: Bloomington, IL
Size: 1001 to 5000 Employees
Founded: 1916
Type of Ownership: Company - Private
Industry: Banking & Lending
Sector: Financial Services
Revenue: $10+ billion (USD)
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 455/900/ dup219
Job Title: Data Platform Engineer File Transfer
Salary Estimate: $106,038 /yr (est.)
Job 

Job Title: Python Software Engineer - Security Data Science
Salary Estimate: $150,000 /yr (est.)
Job Description: Our Team:
We protect Bloomberg.
As part of the Chief Information Security Office - our Security Data Science team is dedicated to identifying, researching, and solving complex information security problems using a data-centric approach.

Working closely with the other security and risk teams - we collect, process, and analyze data generated throughout the company to find new approaches to take on hard security challenges.

What's the Role:
As a Python Data Science Software Engineer, you will hel
Rating: 4.1
Company Name: Bloomberg
4.1
Location: New York, NY
Size: 10000+ Employees
Founded: 1981
Type of Ownership: Company - Private
Industry: Computer Hardware Development
Sector: Information Technology
Revenue: Unknown / Non-Applicable
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Progress: 455/900/ dup228
Job Title: Azure Data Engineer
Salary Estimate: $102,500 /yr (es

In [175]:
first_scrape = jobs
# first_scrape

In [176]:
import pandas as pd
stored_dataframe = pd.read_csv('glassdoor_ds_job_salary.csv')

In [177]:
newly_scraped = pd.DataFrame(first_scrape)
newly_scraped.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Size', 'Founded', 'Type of ownership',
       'Industry', 'Sector', 'Revenue'],
      dtype='object')

In [178]:
new_df = pd.concat([stored_dataframe, newly_scraped], axis=0)
new_df.drop_duplicates()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist V,"$111,362 /yr (est.)","Posting End Date:\nApril 09, 2023\nEmployee Ty...",4.1,Enbridge\n4.1,"Houston, TX",10000+ Employees,1949,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
1,Data Scientist,-1,Data Scientist\nJob Category: Science\nTime Ty...,4.0,CACI\n4.0,"Charlottesville, VA",10000+ Employees,1962,Company - Public,Aerospace & Defense,Aerospace & Defense,$1 to $5 billion (USD)
2,Designer UIX NLP/NLU,"$117,500 /yr (est.)",* W2 Only *\nThe successful candidate will pro...,-1.0,"RightFit Talent Brokers, LLC",Remote,-1,-1,-1,-1,-1,-1
3,Machine Learning Engineer,"$129,337 /yr (est.)",Signal Systems Corporation (SSC) is searching ...,-1.0,Signal Systems Corporation,"Millersville, MD",-1,-1,-1,-1,-1,-1
4,Senior Quality Data Analyst,"$80,968 /yr (est.)","FIRST SHIFT - Draeger Medical Systems, Inc. - ...",3.8,"Draeger Medical Systems, Inc.\n3.8","Andover, MA",10000+ Employees,1889,Company - Public,Health Care Products Manufacturing,Manufacturing,$1 to $5 billion (USD)
...,...,...,...,...,...,...,...,...,...,...,...,...
450,Data Solutions Engineer - Visualization,"$91,687 /yr (est.)",Job Summary:\n\nHealogics is the largest provi...,3.0,"Healogics, Inc\n3.0","Jacksonville, FL",1001 to 5000 Employees,1995,Company - Private,Health Care Services & Hospitals,Healthcare,Unknown / Non-Applicable
451,Data Engineer,$55.00 /hr (est.),RESPONSIBILITIES:\nKforce has a client in Char...,4.0,Kforce\n4.0,"Charlotte, NC",10000+ Employees,1962,Company - Public,Business Consulting,Management & Consulting,$1 to $5 billion (USD)
452,Data Engineer,"$108,000 /yr (est.)",Company Description\n\nCompany Description\nSi...,4.0,Sia Partners\n4.0,"Bellevue, WA",1001 to 5000 Employees,1999,Company - Private,Business Consulting,Management & Consulting,$100 to $500 million (USD)
453,Data Engineer,"$127,500 /yr (est.)",Radancy Data Engineering is seeking a Data Eng...,4.0,Radancy\n4.0,United States,501 to 1000 Employees,2006,Company - Private,Enterprise Software & Network Solutions,Information Technology,$100 to $500 million (USD)


In [179]:
new_df.drop_duplicates(subset=['Job Title', 'Salary Estimate', 'Rating',
       'Company Name', 'Location', 'Size', 'Founded', 'Type of ownership',
       'Industry', 'Sector', 'Revenue'], keep='last')

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist V,"$111,362 /yr (est.)","Posting End Date:\nApril 09, 2023\nEmployee Ty...",4.1,Enbridge\n4.1,"Houston, TX",10000+ Employees,1949,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
1,Data Scientist,-1,Data Scientist\nJob Category: Science\nTime Ty...,4.0,CACI\n4.0,"Charlottesville, VA",10000+ Employees,1962,Company - Public,Aerospace & Defense,Aerospace & Defense,$1 to $5 billion (USD)
2,Designer UIX NLP/NLU,"$117,500 /yr (est.)",* W2 Only *\nThe successful candidate will pro...,-1.0,"RightFit Talent Brokers, LLC",Remote,-1,-1,-1,-1,-1,-1
3,Machine Learning Engineer,"$129,337 /yr (est.)",Signal Systems Corporation (SSC) is searching ...,-1.0,Signal Systems Corporation,"Millersville, MD",-1,-1,-1,-1,-1,-1
4,Senior Quality Data Analyst,"$80,968 /yr (est.)","FIRST SHIFT - Draeger Medical Systems, Inc. - ...",3.8,"Draeger Medical Systems, Inc.\n3.8","Andover, MA",10000+ Employees,1889,Company - Public,Health Care Products Manufacturing,Manufacturing,$1 to $5 billion (USD)
...,...,...,...,...,...,...,...,...,...,...,...,...
450,Data Solutions Engineer - Visualization,"$91,687 /yr (est.)",Job Summary:\n\nHealogics is the largest provi...,3.0,"Healogics, Inc\n3.0","Jacksonville, FL",1001 to 5000 Employees,1995,Company - Private,Health Care Services & Hospitals,Healthcare,Unknown / Non-Applicable
451,Data Engineer,$55.00 /hr (est.),RESPONSIBILITIES:\nKforce has a client in Char...,4.0,Kforce\n4.0,"Charlotte, NC",10000+ Employees,1962,Company - Public,Business Consulting,Management & Consulting,$1 to $5 billion (USD)
452,Data Engineer,"$108,000 /yr (est.)",Company Description\n\nCompany Description\nSi...,4.0,Sia Partners\n4.0,"Bellevue, WA",1001 to 5000 Employees,1999,Company - Private,Business Consulting,Management & Consulting,$100 to $500 million (USD)
453,Data Engineer,"$127,500 /yr (est.)",Radancy Data Engineering is seeking a Data Eng...,4.0,Radancy\n4.0,United States,501 to 1000 Employees,2006,Company - Private,Enterprise Software & Network Solutions,Information Technology,$100 to $500 million (USD)


In [183]:
new_df[new_df['Job Title'] == "Lead Data Engineer"]

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
627,Lead Data Engineer,"$118,255 /yr (est.)",Description:\nCOMPANY OVERVIEW\nArtisan Design...,4.1,Artisan Design Group\n4.1,"Dallas, TX",1001 to 5000 Employees,2016,Company - Private,Construction,"Construction, Repair & Maintenance Services",Unknown / Non-Applicable
672,Lead Data Engineer,"$95,852 /yr (est.)",*InTulsa is partnering with 9b corporation to ...,5.0,inTulsa\n5.0,"Tulsa, OK",1 to 50 Employees,-1,Nonprofit Organization,-1,-1,Unknown / Non-Applicable
676,Lead Data Engineer,"$170,000 /yr (est.)","Join Aya Healthcare, winner of multiple Top Wo...",4.4,Ayahealthcare\n4.4,Oregon,5001 to 10000 Employees,2001,Company - Private,HR Consulting,Human Resources & Staffing,Unknown / Non-Applicable
905,Lead Data Engineer,"$118,255 /yr (est.)",Description:\nCOMPANY OVERVIEW\nArtisan Design...,4.1,Artisan Design Group\n4.1,"Dallas, TX",1001 to 5000 Employees,2016,Company - Private,Construction,"Construction, Repair & Maintenance Services",Unknown / Non-Applicable
944,Lead Data Engineer,"$95,852 /yr (est.)",*InTulsa is partnering with 9b corporation to ...,5.0,inTulsa\n5.0,"Tulsa, OK",1 to 50 Employees,-1,Nonprofit Organization,-1,-1,Unknown / Non-Applicable
948,Lead Data Engineer,"$170,000 /yr (est.)","Join Aya Healthcare, winner of multiple Top Wo...",4.4,Ayahealthcare\n4.4,Oregon,5001 to 10000 Employees,2001,Company - Private,HR Consulting,Human Resources & Staffing,Unknown / Non-Applicable
999,Lead Data Engineer,"$118,329 /yr (est.)",Description:\nCOMPANY OVERVIEW\nArtisan Design...,4.1,Artisan Design Group\n4.1,"Dallas, TX",1001 to 5000 Employees,2016,Company - Private,Construction,"Construction, Repair & Maintenance Services",Unknown / Non-Applicable
266,Lead Data Engineer,"$118,255 /yr (est.)",Description:\nCOMPANY OVERVIEW\nArtisan Design...,4.1,Artisan Design Group\n4.1,"Dallas, TX",1001 to 5000 Employees,2016,Company - Private,Construction,"Construction, Repair & Maintenance Services",Unknown / Non-Applicable
317,Lead Data Engineer,"$95,852 /yr (est.)",*InTulsa is partnering with 9b corporation to ...,5.0,inTulsa\n5.0,"Tulsa, OK",1 to 50 Employees,-1,Nonprofit Organization,-1,-1,Unknown / Non-Applicable
321,Lead Data Engineer,"$170,000 /yr (est.)","Join Aya Healthcare, winner of multiple Top Wo...",4.4,Ayahealthcare\n4.4,Oregon,5001 to 10000 Employees,2001,Company - Private,HR Consulting,Human Resources & Staffing,Unknown / Non-Applicable


In [184]:
repeated_df = new_df[new_df.duplicated(subset=['Job Title', 'Salary Estimate', 'Rating',
       'Company Name', 'Location', 'Size', 'Founded', 'Type of ownership',
       'Industry', 'Sector', 'Revenue'])].sort_values('Job Description')
# repeated_df[repeated_df['Job Title'] == "Lead Data Engineer"]
repeated_df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
944,Lead Data Engineer,"$95,852 /yr (est.)",*InTulsa is partnering with 9b corporation to ...,5.0,inTulsa\n5.0,"Tulsa, OK",1 to 50 Employees,-1,Nonprofit Organization,-1,-1,Unknown / Non-Applicable
905,Lead Data Engineer,"$118,255 /yr (est.)",Description:\nCOMPANY OVERVIEW\nArtisan Design...,4.1,Artisan Design Group\n4.1,"Dallas, TX",1001 to 5000 Employees,2016,Company - Private,Construction,"Construction, Repair & Maintenance Services",Unknown / Non-Applicable
948,Lead Data Engineer,"$170,000 /yr (est.)","Join Aya Healthcare, winner of multiple Top Wo...",4.4,Ayahealthcare\n4.4,Oregon,5001 to 10000 Employees,2001,Company - Private,HR Consulting,Human Resources & Staffing,Unknown / Non-Applicable


In [185]:
updated_scraped = new_df.drop_duplicates(subset=['Job Title', 'Salary Estimate', 'Rating',
       'Company Name', 'Location', 'Size', 'Founded', 'Type of ownership',
       'Industry', 'Sector', 'Revenue'], keep='last')

In [186]:
updated_scraped.to_csv('glassdoor_ds_job_salary.csv', index=False)

In [15]:
stored_dataframe

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist V,"$111,362 /yr (est.)","Posting End Date:\nApril 09, 2023\nEmployee Ty...",4.1,Enbridge\n4.1,"Houston, TX",10000+ Employees,1949,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
1,Data Scientist,-1,Data Scientist\nJob Category: Science\nTime Ty...,4.0,CACI\n4.0,"Charlottesville, VA",10000+ Employees,1962,Company - Public,Aerospace & Defense,Aerospace & Defense,$1 to $5 billion (USD)
2,Data Science/Analysis Research Librarian,-1,Hamilton College invites applications for the ...,4.6,Hamilton College\n4.6,"Clinton, NY",501 to 1000 Employees,1812,College / University,Colleges & Universities,Education,$25 to $100 million (USD)
3,Senior Systems Engineer - Systems Modeling & C...,"$125,000 /yr (est.)",You will have an opportunity to directly impac...,4.1,Cenergy International Services\n4.1,"Boston, MA",10000+ Employees,1907,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
4,Senior Data Scientist - TS Clearance Required,"$131,400 /yr (est.)",Sr Data Scientist (up to 25% profit sharing be...,-1.0,Eighty20 Collective,"Sterling, VA",Unknown,-1,Company - Public,-1,-1,Unknown / Non-Applicable
...,...,...,...,...,...,...,...,...,...,...,...,...
277,Data Scientist - Supply Planning,"$100,638 /yr (est.)",Beam Suntory is Crafting the Spirits that Stir...,3.9,Beam Inc.\n3.9,NaN,5001 to 10000 Employees,2014,Company - Private,Food & Beverage Manufacturing,Manufacturing,$1 to $5 billion (USD)
278,Applied Machine Learning Scientist - Data Tech...,"$205,000 /yr (est.)",Who we are:\nThe Bloomberg Engineering Data Te...,4.1,Bloomberg\n4.1,"Middletown, CT",10000+ Employees,1981,Company - Private,Computer Hardware Development,Information Technology,Unknown / Non-Applicable
279,Senior Data Analyst,$72.50 /hr (est.),Salary: $65-$80/hour\nIndustry: Footwear & App...,4.8,BolyːWelch\n4.8,"Beaverton, OR",1 to 50 Employees,1986,Company - Private,HR Consulting,Human Resources & Staffing,$5 to $25 million (USD)
280,Senior Language Modeling Engineer - Artificial...,"$212,500 /yr (est.)",Bloomberg is a global leader in business and f...,4.1,Bloomberg\n4.1,"Middletown, CT",10000+ Employees,1981,Company - Private,Computer Hardware Development,Information Technology,Unknown / Non-Applicable


In [150]:
second_scrape = jobs
second_scrape

[{'Job Title': 'Data Scientist V',
  'Salary Estimate': '$111,362\xa0/yr (est.)',
  'Job Description': 'Posting End Date:\nApril 09, 2023\nEmployee Type:\nRegular-Full time\nUnion/Non:\nThis is a non-union position\nLife takes energy. The Enbridge Technology + Innovation Lab works with data that powers our products to improve safety and reliability. By working hands-on with ground-breaking technology, the lab pioneers the development of innovative products through small agile teams. Our teams incorporate a variety of multidisciplinary skills, including industrial predictive algorithms, machine learning, and sentiment analysis.\nAs a Data Scientist V, you’ll bring a deep understanding of big data and will help the T+IL to build and enable big data analytics solutions. We build advanced technology products to deliver energy in a safe and reliable way to fuel a rapidly innovating world. Here in our new Technology + Innovation Lab we believe in Agile way of working, focus on our people and

In [171]:
combined_first_second = first_scrape + second_scrape + third_scrape + success_list
len(combined_first_second)

1458

In [165]:
third_scrape = jobs
len(third_scrape)

101

In [209]:
import json
unique_values = set(json.dumps(d) for d in success_list)

print(len(unique_values))
unique_values

146


{'{"Job Title": "AI Artificial Intelligence Developer", "Salary Estimate": "$103,572\\u00a0/yr (est.)", "Job Description": "Genesis10 is seeking an Application Developer for a 12 month contract to hire position with a corporate investment banking company located in Jersey City, NJ. This is a hybrid position and requires being on site 3 days per week.\\n\\nSummary:\\nThis developer is in the conversational artificial intelligence space. As a Conversational AI Lead, you will join our Cognitive Automation team. You will be responsible for every stage of a Conversational AI project - lead the architecture, development, and implementation of conversational AI projects from research and design to conversation scripting and testing.\\n\\nResponsibilities:\\nOwn every stage of a Conversational AI project - lead the architecture, development, and implementation of conversational AI projects from research and design to conversation scripting and testing.\\n\\n\\n\\n\\n\\n\\n\\nRanked a Top Staff

In [174]:
success_list = jobs
len(success_list)
success_set = set(json.dumps(d) for d in success_list)
len(success_set)

146

In [123]:
from collections import defaultdict
def frequencies(lst):
    freq = defaultdict(int)
    for val in lst:
        freq[val] += 1
    return dict(freq)



In [125]:
list = [1, 2, 3]
frequencies(list)

{1: 1, 2: 1, 3: 1}


In [189]:
success_lst = [json.dumps(d) for d in success_list]

In [195]:
successs_dict = frequencies(success_lst)

In [199]:
type(successs_dict)

dict

In [200]:
for key, value in successs_dict.items():
    print(value)

25
25
25
22
25
25
25
12
25
25
24
11
25
25
25
25
25
19
25
5
2
25
25
12
25
1
1
24
3
1
16
1
12
7
1
5
21
8
5
12
9
2
2
16
2
6
14
4
2
13
4
1
2
2
2
2
1
1
2
1
2
5
12
2
1
1
2
1
1
2
1
2
2
1
1
1
1
1
1
2
10
1
2
1
1
3
1
3
1
1
2
3
1
2
2
2
2
1
1
1
2
1
1
2
1
1
1
2
1
2
2
2
1
2
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
2
17
1
3
1
3
2
2
1
1


In [202]:
successs_dict

{'{"Job Title": "Senior Python Data Scientist", "Salary Estimate": -1, "Job Description": "Overview:\\nOasis Systems has an exciting opportunity for a Senior Python Data Scientist to support the Commodity Futures Trading Commission (CFTC) in Washington, DC. The contractor shall provide support in data analytics (including descriptive and predictive analytics and data mining), large data manipulation, data visualization, and the use of data analytic tools. Activities that the contractor may be tasked to perform include gathering requirements from stakeholders, conducting financial data analysis in a Big Data environment, and creating reports and documentation.\\n\\nLOCATION: Washington, DC\\nJOB STATUS: Full-Time\\nTRAVEL: N/A\\nREQUIRED QUALIFICATIONS (Education, Certifications, Experience, Skills)\\nSECURITY CLEARANCE: Ability to pass a Special Agreement Check (SAC) and a National Agency Check and Inquiries (NACI) Background Investigation and U.S. Citizenship is required for all appli

In [ ]:
first_scrape.extend(second_scrape)

In [59]:
response = requests.get(target_url)
if response.status_code == 200:
    print("Success")
else:
    print("Failure")
soup = BeautifulSoup(response.content, 'html.parser')
allJobsContainer = soup.find("ul")

#allJobs = allJobsContainer.find_all("li")

Failure


In [60]:
response.status_code

403

In [57]:
allJobsContainer

In [134]:
driver = webdriver.Chrome()
driver.get("https://www.glassdoor.com/Job/united-states-data-scientist-jobs-SRCH_IL.0,13_IN1_KO14,28_IP2.htm?includeNoSalaryJobs=true&pgc=AB4AAYEAHgAAAAAAAAAAAAAAAf8xnJwAQwEBAQcCAHscnY62Qk942mPGLgWfOJanLoCN0GWcjSMiOGPasDuVRUW2qPToi7eE38gBBUcX%2B491ifLlRBwp%2BqUxcl8AAA%3D%3D")

    # Get element with tag name 'div'
element = driver.find_element(By.CSS_SELECTOR, '[alt="Close"]').click()
job_buttons = driver.find_elements(By.CSS_SELECTOR, '[data-test="jobListing"]')  #jl for Job Listing. These are the buttons we're going to click.
for job_button in job_buttons: 
    try:
        driver.find_element(By.CSS_SELECTOR, '[alt="Close"]').click()  #clicking to the X.
    except NoSuchElementException:
        pass
    job_button.click()
    try:
        driver.find_element(By.CSS_SELECTOR, '[alt="Close"]').click()  #clicking to the X.
    except NoSuchElementException:
        pass

            
    collected_successfully = False

    while not collected_successfully:
        try:
            try:
                company_name = driver.find_element(By.CSS_SELECTOR, '[data-test="employerName"]').text
            except: 
                company_name = driver.find_element(By.CSS_SELECTOR, '[class="d-flex justify-content-between"]').text
            company_name = driver.find_element(By.CSS_SELECTOR, '[data-test="employerName"]').text
            location = driver.find_element(By.CSS_SELECTOR, '[data-test="location"]').text
            job_title = driver.find_element(By.CSS_SELECTOR, '[data-test="job-link"]').text
            driver.find_element(By.CSS_SELECTOR, '[class="css-t3xrds e856ufb4"]').click()
            job_description = driver.find_element(By.CSS_SELECTOR, '[id="JobDescriptionContainer"]').text
            collected_successfully = True
        except:
            time.sleep(5)

        try:
            salary_estimate = driver.find_element(By.CSS_SELECTOR, '[data-test="detailSalary"]').text
        except NoSuchElementException:
            salary_estimate = -1 #You need to set a "not found value. It's important."

        try:
            rating = driver.find_element(By.CSS_SELECTOR, '[data-test="detailRating"]').text
        except NoSuchElementException:
            rating = -1 #You need to set a "not found value. It's important."

    overview = driver.find_elements(By.CSS_SELECTOR, '[class="css-i9gxme e1pvx6aw2"]')
    break
# #for each in overview:
# size = overview[0].text
# founded = overview[1].text
# type_of_ownership = overview[2].text
# industry = overview[3].text
# sector = overview[4].text
# revenue = overview[5].text
    #print(each.text)
driver.maximize_window()


In [135]:
job_description

"FLSA Status\nExempt\nSalary Range\n$135,000-$145,000 + incentive bonus of up to 20% of base salary\nExact compensation may vary based on skills, experience, and location.\nJob Family\nData Strategy and Services\n\n**Fusion Medical Staffing does not offer Visa Sponsorship at this time**\n\nOverview: Fusion Medical Staffing is searching for a talented Manager of Data Engineering and Science to join our growing Data Strategy and Services team. In this role, you will play a pivotal role for continued design, development and execution of a Centralized Data Warehouse while managing the flow of data and providing direction for deep analysis and leading Fusion’s Data Science Projects. You will be responsible for maintaining and growing an enterprise-wide data infrastructure that will supply the business with high quality, certified data sets and a structure that will support and enable the future of the business for Data Analytics, Machine Learning, AI, and Data Sciences. You will work with a

In [87]:
company_name

'Enbridge\n4.1'

In [88]:
location

'Houston, TX'

In [89]:
job_title

'Data Scientist V'

In [90]:
job_description

'Posting End Date:\nApril 09, 2023\nEmployee Type:\nRegular-Full time\nUnion/Non:\nThis is a non-union position\nLife takes energy. The Enbridge Technology + Innovation Lab works with data that powers our products to improve safety and reliability. By working hands-on with ground-breaking technology, the lab pioneers the development of innovative products through small agile teams. Our teams incorporate a variety of multidisciplinary skills, including industrial predictive algorithms, machine learning, and sentiment analysis.\nAs a Data Scientist V, you’ll bring a deep understanding of big data and will help the T+IL to build and enable big data analytics solutions. We build advanced technology products to deliver energy in a safe and reliable way to fuel a rapidly innovating world. Here in our new Technology + Innovation Lab we believe in Agile way of working, focus on our people and value craftsmanship, in a ‘work is play’ environment where we offer a flexible, fun, and modern workin

In [91]:
salary_estimate

'$94K - $132K (Glassdoor est.)'

In [92]:
rating

'4.1'

In [95]:
size

'10000+ Employees'

In [96]:
founded

'1949'

In [97]:
type_of_ownership

'Company - Public'

In [98]:
industry

'Energy & Utilities'

In [99]:
sector

'Energy, Mining & Utilities'

In [ ]:
revenue

In [108]:
job_buttons

[<selenium.webdriver.remote.webelement.WebElement (session="b3bc03238d283218d55ac7cd65840d5d", element="c8d23342-6b79-46eb-9451-7a0b9839debe")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3bc03238d283218d55ac7cd65840d5d", element="087f8e60-392d-441a-b278-488151be0aba")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3bc03238d283218d55ac7cd65840d5d", element="c909546e-d596-4b36-8a1a-6282bbcc5475")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3bc03238d283218d55ac7cd65840d5d", element="a2102519-b736-4906-b0f7-943039733798")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3bc03238d283218d55ac7cd65840d5d", element="77fb654c-7d41-4537-a0b2-56e5942b948e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3bc03238d283218d55ac7cd65840d5d", element="3e426682-577d-4f78-9345-1b24d393d8e7")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b3bc03238d283218d55ac7cd65840d5d", element="909a9eab-9409-4c2c-9973-82

In [117]:
type(job_title)

str

In [205]:
import pandas as pd
df = pd.DataFrame(combined_first_second)
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist V,"$111,362 /yr (est.)","Posting End Date:\nApril 09, 2023\nEmployee Ty...",4.1,Enbridge\n4.1,"Houston, TX",10000+ Employees,1949,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
1,Data Scientist,-1,Data Scientist\nJob Category: Science\nTime Ty...,4.0,CACI\n4.0,"Charlottesville, VA",10000+ Employees,1962,Company - Public,Aerospace & Defense,Aerospace & Defense,$1 to $5 billion (USD)
2,Data Science/Analysis Research Librarian,-1,Hamilton College invites applications for the ...,4.6,Hamilton College\n4.6,"Clinton, NY",501 to 1000 Employees,1812,College / University,Colleges & Universities,Education,$25 to $100 million (USD)
3,Senior Systems Engineer - Systems Modeling & C...,"$125,000 /yr (est.)",You will have an opportunity to directly impac...,4.1,Cenergy International Services\n4.1,"Boston, MA",10000+ Employees,1907,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
4,Senior Data Scientist - TS Clearance Required,"$131,400 /yr (est.)",Sr Data Scientist (up to 25% profit sharing be...,-1,Eighty20 Collective,"Sterling, VA",Unknown,-1,Company - Public,-1,-1,Unknown / Non-Applicable
...,...,...,...,...,...,...,...,...,...,...,...,...
1453,Applied Statistician - Data Technologies,"$205,000 /yr (est.)",Who we are:\nThe Bloomberg Engineering Data Te...,4.1,Bloomberg\n4.1,"New York, NY",10000+ Employees,1981,Company - Private,Computer Hardware Development,Information Technology,Unknown / Non-Applicable
1454,Pipeline Integrity Risk Assessment & Data Scie...,"$148,650 /yr (est.)","About Us\n\nAt SoCalGas, we believe that every...",4.4,SoCalGas\n4.4,"Los Angeles, CA",5001 to 10000 Employees,1929,Subsidiary or Business Segment,Energy & Utilities,"Energy, Mining & Utilities",$1 to $5 billion (USD)
1455,"Data Engineer, Senior Consultant","$97,622 /yr (est.)",Job Family:\nData Science Consulting\n\nTravel...,3.7,Guidehouse\n3.7,"Huntsville, AL",1001 to 5000 Employees,2018,Company - Private,Business Consulting,Management & Consulting,Unknown / Non-Applicable
1456,Senior Decision Science Manager,-1,Position Overview:\nCar Capital Technologies i...,2.4,CAR CAPITAL CORPORATION\n2.4,"Grapevine, TX",1 to 50 Employees,-1,Company - Private,-1,-1,Unknown / Non-Applicable


In [207]:
df_no_dup = df.drop_duplicates()

In [208]:
df_no_dup.to_csv('glassdoor_ds_job_salary.csv', index=False)

In [16]:
newly_scraped

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist V,"$111,371 /yr (est.)","Posting End Date:\nApril 09, 2023\nEmployee Ty...",4.1,Enbridge\n4.1,"Houston, TX",10000+ Employees,1949,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
1,Senior Data Scientist - TS Clearance Required,"$131,400 /yr (est.)",Sr Data Scientist (up to 25% profit sharing be...,-1,Eighty20 Collective,"Sterling, VA",Unknown,-1,Company - Public,-1,-1,Unknown / Non-Applicable
2,Senior Systems Engineer - Systems Modeling & C...,"$125,000 /yr (est.)",You will have an opportunity to directly impac...,4.1,Cenergy International Services\n4.1,"Boston, MA",10000+ Employees,1907,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
3,Data Engineer,"$110,000 /yr (est.)",NATO Data Engineer\nRequirements:\nTs/sci or s...,-1,"Bluemont Technology & Research, Inc.","Norfolk, VA",-1,-1,-1,-1,-1,-1
4,Machine Learning Engineer,"$130,000 /yr (est.)",Job Description Overview:\nVectrus seeks a res...,3.9,Vectrus\n3.9,"King George, VA",10000+ Employees,1945,Company - Public,Aerospace & Defense,Aerospace & Defense,Unknown / Non-Applicable
...,...,...,...,...,...,...,...,...,...,...,...,...
145,Data Scientist - RWD,"$150,000 /yr (est.)",Job Summary:\nWe are seeking an experienced Da...,-1,Norstella,Remote,-1,-1,-1,-1,-1,-1
146,Data Scientist,"$120,000 /yr (est.)",Gloo was founded as a mission-driven organizat...,3.6,Gloo\n3.6,"Boulder, CO",51 to 200 Employees,2013,Company - Private,Enterprise Software & Network Solutions,Information Technology,$1 to $5 million (USD)
147,Sr. Data Scientist,"$107,469 /yr (est.)","Become a Part of the NIKE, Inc. Team\nNIKE, In...",4.1,Nike\n4.1,"Beaverton, OR",10000+ Employees,1972,Company - Public,Consumer Product Manufacturing,Manufacturing,$10+ billion (USD)
148,Data Scientist,"$108,421 /yr (est.)","With a focus on Mobility, Operational Excellen...",3.9,Nissan\n3.9,"Franklin, TN",10000+ Employees,1933,Company - Public,Transportation Equipment Manufacturing,Manufacturing,$1 to $5 billion (USD)


After webscraping the data, we put it into a csv file.


DATA CLEANING SECTION

In [39]:
import pandas as pd

In [40]:
df = pd.read_csv('glassdoor_ds_job_salary.csv')

In [41]:
df

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
0,Data Scientist V,"$111,362 /yr (est.)","Posting End Date:\r\nApril 09, 2023\r\nEmployee Type:\r\nRegular-Full time\r\nUnion/Non:\r\nThis is a non-union position\r\nLife takes energy. The Enbridge Technology + Innovation Lab works with data that powers our products to improve safety and reliability. By working hands-on with ground-breaking technology, the lab pioneers the development of innovative products through small agile teams. Our teams incorporate a variety of multidisciplinary skills, including industrial predictive algorithms, machine learning, and sentiment analysis.\r\nAs a Data Scientist V, you’ll bring a deep understanding of big data and will help the T+IL to build and enable big data analytics solutions. We build advanced technology products to deliver energy in a safe and reliable way to fuel a rapidly innovating world. Here in our new Technology + Innovation Lab we believe in Agile way of working, focus on our people and value craftsmanship, in a ‘work is play’ environment where we offer a flexible, fun, and modern working style to support individual needs.",4.1,Enbridge\r\n4.1,"Houston, TX",10000+ Employees,1949,Company - Public,Energy & Utilities,"Energy, Mining & Utilities",$10+ billion (USD)
1,Data Scientist,-1,"Data Scientist\r\nJob Category: Science\r\nTime Type: Full time\r\nMinimum Clearance Required to Start: TS/SCI\r\nEmployee Type: Regular\r\nPercentage of Travel Required: Up to 10%\r\nType of Travel: Local\r\nCACI is seeking a Mathematics / Statistical Scientist (Mid) to perform and manage the production of various products in coordination with project and program leadership. This position will support development and application of modeling and simulation tools and algorithms and data mining, analysis, aggregation, fusion, and visualization to process, analyze, and integrate disparate data sources facilitating the production and dissemination of S&TI and GMI relating to forecasting of future foreign ground forces research, development, weapon systems, employment, and operations. The candidate shall integrate statistical data analysis techniques with raw and finished intelligence and geospatial data and services to produce analysis aligned with research objectives. The work involves ensuring data outputs and analysis are available in actionable, standard, and usable forms and utilizing the application of intelligence principles in concert with a practical knowledge of information technology to provide technical mission support.\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\nCompany Overview: At CACI, you will have the opportunity to make an immediate impact by providing information solutions and services in support of national security missions and government transformation for Intelligence, Defense, and Federal Civilian customers. CACI is an Equal Opportunity/Affirmative Action Employer. All qualified applicants will receive consideration for employment without regard to race, color, religion, sex, sexual orientation, gender identity, national origin, disability, status as a protected veteran, or any other protected characteristic.",4.0,CACI\r\n4.0,"Charlottesville, VA",10000+ Employees,1962,Company - Public,Aerospace & Defense,Aerospace & Defense,$1 to $5 billion (USD)
2,Data Science/Analysis Research Librarian,-1,"Hamilton College invites applications for the position of Data Science/Analysis Research Librarian in the Library and Information Technology Services (LITS). The Data Science/Analysis Research Librarian will provide research support and introduce cutting-edge data analysis tools and methodologies to LITS patrons as a member of the Research and Instructional Design (R&ID) Team where research and technology experts work together to support faculty and students with research and teaching. The R&ID Team provides an inclusive, respectful, and responsive service point, works co

In [62]:
#we first check for duplicates
print(f"There are {df.duplicated().sum()} duplicates")

# remove duplicates
df.drop_duplicates(inplace=True)

There are 0 duplicates


C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_69876\2120758634.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [44]:
# check for missing values
df.isna().sum()

Job Title            0
Salary Estimate      0
Job Description      0
Rating               0
Company Name         0
Location             8
Size                 0
Founded              0
Type of ownership    0
Industry             0
Sector               0
Revenue              0
dtype: int64

In [45]:
#remove unnecessary columns
df.drop(['Founded', 'Type of ownership'], axis=1, inplace=True)

We are now going to handle the most important part of hour dataset: salary estimate

In [46]:
#remove all rows that have a missing estimated salary. We need labeled examples in order to build our model!

# We first Count the number of rows with -1 as salary estimate (which means it is missing)
count = df[df['Salary Estimate'] == '-1'].shape[0]
print(f'There are {count} rows with a missing salary estimate')

#remove those rows
df = df[df['Salary Estimate'] != '-1']

There are 57 rows with a missing salary estimate


In [49]:
import re

#Let's check if all the salaries have the same format. For example: $131,400 /yr (est.)


# Define regular expression pattern to check format
pattern = r'\$\d{1,3}(,\d{3})*\s?\/yr\s?\(est.\)'

# Check if all values in the Salary Estimate column have the expected format
matches = df['Salary Estimate'].apply(lambda x: bool(re.match(pattern, x)))

not_matching = df[~matches]
print(f'Number of Vvalues in the Salary Estimate column have do not have the expected format: {len(not_matching)}')
not_matching

Number of Vvalues in the Salary Estimate column have do not have the expected format: 14


Job Title  \
14                     Data Mining Center Journeyman Electrician   
22   Sr Data Analyst-Clinical Evidence Strategy & Implementation   
36               Contract Senior Statistical Programmer (Remote)   
81               AI/ML Autonomous Vehicle Software Video Analyst   
89                                                  Statistician   
98                                Data Scientist_Computer Vision   
134                                               DATA SCIENTIST   
136                                          Senior Statistician   
146                                               Data Scientist   
241                                Senior Statistical Programmer   
250                                               Data Scientist   
279                                          Senior Data Analyst   
300                                               Data Scientist   
402                                               Data Scientist   

        Salary Estimate  \
14    $33.00 /hr (est.)   
22    $35.50 /hr (est.)   
36   $100.00 /hr (est.)   
81    $27.00 /hr (est.)   
89    $36.00 /hr (est.)   
98    $85.00 /hr (est.)   
134   $7,917 /mo (est.)   
136  $115.00 /hr (est.)   
146   $70.00 /hr (est.)   
241   $89.00 /hr (est.)   
250   $76.63 /hr (est.)   
279   $72.50 /hr (est.)   
300   $72.50 /hr (est.)   
402   $72.50 /hr (est.)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [59]:
# We realize that some salaries are expressed at a hourly rate, instead of a yearly rate.


# Identify rows with hourly salary values
hourly_mask = df['Salary Estimate'].str.contains('/hr')

# Convert hourly salary values to yearly salary values
df.loc[hourly_mask, 'Salary Estimate'] = df.loc[hourly_mask, 'Salary Estimate'].apply(lambda x: str(float(x[1:x.index('/')].replace(',', '')) * 2080) + ' /yr (est.)')


# Remove any non-numeric characters and convert to numeric
df['Salary Estimate'] = df['Salary Estimate'].apply(lambda x: re.sub(r'[^0-9]', '', x))

df['Salary Estimate'] = pd.to_numeric(df['Salary Estimate'])

#We now have in our salary estimate column yearly salaries expressed as integers

C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_69876\878150041.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Salary Estimate'] = df['Salary Estimate'].str.replace('$', '').str.replace(',', '').str.replace('/yr (est.)', '')
C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_69876\878150041.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Salary Estimate'] = df['Salary Estimate'].str.replace('$', '').str.replace(',', '').str.replace('/yr (est.)', '')
C:\Users\Lorenzo\AppData\Local\Temp\ipykernel_69876\878150041.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [61]:
print(df.dtypes)

Job Title           object
Salary Estimate      int64
Job Description     object
Rating             float64
Company Name        object
Location            object
Size                object
Industry            object
Sector              object
Revenue             object
dtype: object
